# KILT-9-HandsOnAnalysis_HowManyTopK

- NQ(KILT) dev FiD with BM25 negative passages
    - Method3. '00011011000'  -> 
    <span style="color:blue">0</span><span style="color:blue">0</span><span style="color:blue">0</span><span style="color:red">1</span></span><span style="color:red">1</span>0<span style="color:red">1</span></span><span style="color:red">1</span>000  (__0.73651__)  

- NQ dev with FiD retrieved passages
    - Method3. '00011011000'  -> 
<span style="color:blue">0</span><span style="color:blue">0</span><span style="color:blue">0</span><span style="color:red">1</span></span><span style="color:red">1</span>0<span style="color:red">1</span></span><span style="color:red">1</span>000  (__0.57839__)     

- Both cases show that including consecutive 0s improved the overall output
- So Questions : How many consecutive 0s are needed?
    - NQ(KILT) dev FiD with BM25 negative passages
    - only 5 passages -> just checking 0s em Pattern

In [35]:
from collections import Counter
import numpy as np
import pathlib
from pprint import pprint
import pandas as pd
from util import utils
import re
import os

In [26]:
input_file = utils.open_json('/data/philhoon-relevance/FiD/results/KILT_BM25_NQ/incremental_result/pos1_ctx5.json')

In [27]:
pprint(input_file[0])

{'answers': ['the therefore sign',
             'therefore sign',
             'the therefore sign ( ∴ ) is generally used before a logical '
             'consequence , such as the conclusion of a syllogism',
             'a logical consequence , such as the conclusion of a syllogism'],
 'conversion': 0,
 'ctxs': [{'psg_id': 3894652,
           'text': 'Therefore sign In logical argument and mathematical proof, '
                   'the therefore sign (∴) is generally used before a logical '
                   'consequence, such as the conclusion of a syllogism. The '
                   'symbol consists of three dots placed in an upright '
                   'triangle and is read "therefore". It is encoded at . While '
                   'it is not generally used in formal writing, it is used in '
                   'mathematics and shorthand. It is complementary to . '
                   'History. According to Cajori, "A History of Mathematical '
                   'Notations," Johan

### format
```python
{
    'id' : str()
    'answers' : list()
    'ctxs' : list(dict)
    'questions' : str()
}
```

In [28]:
sample_size = 5
total = 0
num_zero_lst = []

In [29]:
for inst_ in input_file:
    pattern = inst_['em_pattern']
    if pattern.startswith('0') and pattern != '0' * sample_size:
#         print(pattern)
        result = re.search("^0+", pattern)
#         print(pattern)
#         print(result)
#         p
#         print('-----')
        total += 1
        num_zero_lst.append(result.span()[-1])
#     print(pattern)
# print(total)

In [30]:
print(total)

153


In [31]:
zero_array = np.array(num_zero_lst)

In [36]:
counts = Counter(num_zero_lst)

In [37]:
counts

Counter({2: 48, 1: 64, 4: 16, 3: 25})

In [32]:
len(zero_array)

153

In [33]:
df_describe = pd.DataFrame(zero_array)
df_describe.describe()

,0
count,153.000000
mean,1.954248
std,1.002233
min,1.000000
25%,1.000000
50%,2.000000
75%,3.000000
max,4.000000


#### Test Code 

### Checking on Final Total Trials 12

    1st Trial - Options1(Remove only Damaging) + Strict Positive + Strict Damaging (checked)
    2nd Trial - Options1(Remove only Damaging) + Strict Positive + Naive Damaging (checked)
    3rd Trial - Options1(Remove only Damaging) + Naive Positive + Strict Damaging (checked)
    4th Trial - Options1(Remove only Damaging) + Naive Positive + Naive Damaging (checked)
    
    5th Trial - Options2(Remove damaging + irrelevant) + Strict Positive + Strict Damaging (checked)
        - The one also represents other three cases
            - Options2(Remove damaging + irrelevant) + Strict Positive + Naive Damaging 
            - Options2(Remove damaging + irrelevant) + Naive Positive + Strict Damaging (checked)
            - Options2(Remove damaging + irrelevant) + Naive Positive + Naive Damaging 
    
    6th Trial - Options3(Removes damaging + relevant) + Strict Positive + Strict Damaging (checked)
    7th Trial - Options3(Removes damaging + relevant) + Strict Positive + Naive Damaging (checked)
    8th Trial - Options3(Removes damaging + relevant) + Naive Positive + Strict Damaging (checked)
    9th Trial - Options3(Removes damaging + relevant) + Naive Positive + Naive Damaging (checked)
    
    10th Trial - Options4(Removes damaging + irrelevant + relevant) + Strict Positive (checked)
        - The one represents cases 
            - Options4(Removes damaging + irrelevant + relevant) + Strict Positive + Strict Damaging (checked)
            - Options4(Removes damaging + irrelevant + relevant) + Strict Positive + Naive Damaging
        - almost equal to "pos1_ctx1" e.g.) 01011 -> 2 strict positive cases
        - "pos1_ctx1" = baseline could be our baselines
        
    11th Trial - Options4(Removes damaging + irrelevant + relevant) + Naive Positive (checked)  
        - The one represents cases 
            - Options4(Removes damaging + irrelevant + relevant) + Naive Positive + Strict Damaging
            - Options4(Removes damaging + irrelevant + relevant) + Naive Positive + Naive Damaging (checked)

# Implementation

In [4]:
def build_data(input_file, option, option_p, option_d):
    '''
    input_file : incremental inference result from FiD from KILT-5-1
        path : /data/philhoon-relevance/FiD/results/KILT_BM25_NQ/incremental_result/pos1_ctx5.json
        
    output : FiD input json format
    
    option(required) : removing strategies
        op1 : removes damages only
        op2 : removes damaging + irrelevant
        op3 : removes damaging + relevant
        op4 : removes damaging + irrelevant + relevant
        
    option_p(required) : positive passage selection options
        strict : strict positive
            e.g.) 11 pattern 
                1st '1' is positive, 2nd '1' is relevant
        naive : naive positive
            e.g.) 11 pattern 
                1st '1' is positive, 2nd '1' is positive
                
    option_d(required) : damaging passage selection options
        strict : strict negative
            e.g.) A00 pattern 
                if there is at least one '1' occurred in A, 2nd '0' is irrelevant
        naive : naive damaging
            e.g.) A00 pattern 
                if there is at least one '1' occurred in A, 2nd '0' is damaging
    
    '''
    
    output_format = []

    # 'strict', 'naive'  
    # option_p = 'naive'
    # option_d = 'naive'
    # option = 'op4'

    for instance in input_file:
        template_dict = {}
        template_dict['id'] = instance['id']
        template_dict['answers'] = instance['answers']
        template_dict['question'] = instance['question']
        template_dict['em_pattern'] = instance['em_pattern']

        em_pattern = instance['em_pattern']

        # when there is at least one EM in the accumulated inference
        if em_pattern != '00000':   
            new_ctx = []

            # relevant vs positive
            positve_ctx_lst = []
            relevant_ctx_lst = []

            # irrelevant vs damaging
            damaging_ctx_lst = []
            irrelevant_ctx_lst = []


            for idx_, ctx in enumerate(instance['ctxs']):

                # checking current em
                cur_em = em_pattern[idx_]
                pre_em_pattern = em_pattern[:idx_]


                # first 1 : positive
                if not pre_em_pattern and cur_em == '1':
                    positve_ctx_lst.append(ctx)

                # first 0 : irrelevant
                elif not pre_em_pattern and cur_em == '0':
                    irrelevant_ctx_lst.append(ctx)
                    
                # 01 pattern : positive 
                elif pre_em_pattern and pre_em_pattern[-1] == '0' and cur_em == '1':
                    positve_ctx_lst.append(ctx)

                # 10 pattern : damaging
                elif pre_em_pattern and pre_em_pattern[-1] == '1' and cur_em == '0':
                    damaging_ctx_lst.append(ctx)

                # 11 pattern : Strict Positive(relevant) or Naive Positive(positive)
                elif pre_em_pattern and pre_em_pattern[-1] == '1' and cur_em == '1':
                    if option_p == 'strict':
                        relevant_ctx_lst.append(ctx)

                    elif option_p == 'naive':
                        positve_ctx_lst.append(ctx)

                    else:
                        print('option_p should be either \'strict\' or \'naive\'')
                        return 

                # 00 pattern : Strict Damaging(irrelevant) or Naive Damaging(damaging) 
                elif pre_em_pattern and pre_em_pattern[-1] == '0' and cur_em == '0':
                    # if '1' does not occured in A, currnet passage is irrelevant
                    if not '1' in pre_em_pattern:
                        irrelevant_ctx_lst.append(ctx)

                    # if '1' occurred in A, 
                    else:
                        # strict : consider it as irrelevnat 
                        if option_d == 'strict':
                            irrelevant_ctx_lst.append(ctx)

                        # naive : consider it as damaging 
                        elif option_d == 'naive':
                            damaging_ctx_lst.append(ctx)

                        else:
                            print('option_p should be either \'strict\' or \'naive\'')
                            return 

            # op1 removes damages only
            if option == 'op1':
                new_ctx.extend(positve_ctx_lst)
                new_ctx.extend(relevant_ctx_lst)
                new_ctx.extend(irrelevant_ctx_lst)


            # op2 removes damaging + irrelevant
            elif option == 'op2':
                new_ctx.extend(positve_ctx_lst)
                new_ctx.extend(relevant_ctx_lst)

            # op3 : Removes damaging + relevant
            elif option == 'op3':
                new_ctx.extend(positve_ctx_lst)
                new_ctx.extend(irrelevant_ctx_lst)

            # op4 : Removes damaging + irrelevant + relevant
            elif option == 'op4':
                new_ctx.extend(positve_ctx_lst)

            else:
                print('option should be op1, op2, op3, op4')
                return 

            template_dict['ctxs'] = new_ctx
            output_format.append(template_dict)

        # when there is no EM in the accumulated inference
        else:
            template_dict['ctxs']= instance['ctxs']
            output_format.append(template_dict)
    
    print('==============instance finished======================')
    return output_format
        


In [6]:
option_dict = {
    'op1' : 'remove_damage',
    'op2' : 'remove_damage_irrelevant',
    'op3' : 'remove_damage_relevant',
    'op4' : 'remove_damage_irrelevant_relevant',
}
# option = 'op4'

In [7]:
option_p_dict = {
    'strict' : 'strict_positive',
    'naive' : 'naive_positive',
}
# option_p = 'strict'

In [8]:
option_d_dict = {
    'strict' : 'strict_damaging',
    'naive' : 'naive_damaging',
}
# option_d = 'strict'

In [9]:
input_file = utils.open_json('/data/philhoon-relevance/FiD/results/KILT_BM25_NQ/incremental_result/pos1_ctx5.json')
output_path = '/data/philhoon-relevance/FiD/open_domain_data/NQ_KILT_BM25_SELECTION'

In [10]:
for o_ in option_dict.keys():
    for op in option_p_dict.keys():
        for od in option_d_dict.keys():
            option = o_
            option_p = op
            option_d = od
            
            filename = f'{option_p_dict[option_p]}_{option_d_dict[option_d]}_{option_dict[option]}.json'
            
            output_file = os.path.join(output_path, filename)
            output_format = build_data(input_file, option, option_p, option_d)
            
            utils.save_json(output_format, output_file)
            print(f'{filename} save on \n {output_path}')
#             print(f'option : {option}')
#             print(f'option_p : {option_p}')
#             print(f'option_d : {option_d}')
#             print(f'filename : {filename}')
#             print(f'output_file : {output_file}')
            

==============instance finished======================
strict_positive_strict_damaging_remove_damage.json save on 
 /data/philhoon-relevance/FiD/open_domain_data/NQ_KILT_INCRE_TEST
==============instance finished======================
strict_positive_naive_damaging_remove_damage.json save on 
 /data/philhoon-relevance/FiD/open_domain_data/NQ_KILT_INCRE_TEST
==============instance finished======================
naive_positive_strict_damaging_remove_damage.json save on 
 /data/philhoon-relevance/FiD/open_domain_data/NQ_KILT_INCRE_TEST
==============instance finished======================
naive_positive_naive_damaging_remove_damage.json save on 
 /data/philhoon-relevance/FiD/open_domain_data/NQ_KILT_INCRE_TEST
==============instance finished======================
strict_positive_strict_damaging_remove_damage_irrelevant.json save on 
 /data/philhoon-relevance/FiD/open_domain_data/NQ_KILT_INCRE_TEST
==============instance finished======================
strict_positive_naive_damaging_remove_d